First, data handling, for proper
data reading.

In [ ]:
SHOW GLOBAL VARIABLES LIKE 'local_infile';
SET GLOBAL local_infile = 'ON';

-- (0) Schema selection
DROP DATABASE IF EXISTS sql_olist;
CREATE DATABASE sql_olist; 
USE sql_olist;

-- (1) Read Customers
DROP TABLE  IF EXISTS customers; 
CREATE TABLE customers (
    customer_id TEXT,
    customer_unique_id TEXT,
    customer_zip_code_prefix TEXT,
    customer_city TEXT,
    customer_state TEXT
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_customers_dataset.csv' 
INTO TABLE customers 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
-- LINES TERMINATED BY '\r\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.customers;

-- (2) Read geolocation dataset
DROP TABLE  IF EXISTS geolocation; 
CREATE TABLE geolocation (
    geolocation_zip_code_prefix TEXT,
    geolocation_lat DOUBLE,
    geolocation_lng DOUBLE,
    geolocation_city TEXT,
    geolocation_state TEXT
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_geolocation_dataset.csv' 
INTO TABLE geolocation 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
-- LINES TERMINATED BY '\r\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.geolocation;

-- (3) sellers dataset
DROP TABLE  IF EXISTS sellers; 
CREATE TABLE sellers (
    seller_id TEXT,
    seller_zip_code_prefix BIGINT,
    seller_city TEXT,
    seller_state TEXT
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_sellers_dataset.csv' 
INTO TABLE sellers 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.sellers;

-- (4) read prducts dataset
DROP TABLE  IF EXISTS products; 
CREATE TABLE products (
    product_id TEXT,
    product_category_name TEXT,
    product_name_lenght INT,
    product_description_lenght INT,
    product_photos_qty INT,
    product_weight_g INT,
    product_lenght_cm INT,
    product_height_cm INT,
    product_width_cm INT
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_products_dataset.csv' 
INTO TABLE products
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


SELECT COUNT(*) 
FROM sql_olist.products;

-- (5) read orders dataset
DROP TABLE  IF EXISTS orders; 
CREATE TABLE orders (
    order_id TEXT,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_orders_dataset.csv' 
INTO TABLE orders 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.orders;

-- (6) read order_items_dataset
DROP TABLE  IF EXISTS order_items; 
CREATE TABLE order_items (
    order_id TEXT,
    order_item_id INT,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price DOUBLE,
    freight_value DOUBLE
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_order_items_dataset.csv' 
INTO TABLE order_items 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.order_items;

-- (7) read order payments dataset
DROP TABLE  IF EXISTS order_payments; 
CREATE TABLE order_payments (
    order_id  TEXT,
    payment_sequential INT,
    payment_type TEXT,
    payment_installments INT,
    payment_value DOUBLE
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_order_payments_dataset.csv' 
INTO TABLE order_payments 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


SELECT COUNT(*) 
FROM sql_olist.order_payments;

-- (8) read order reviews dataset
DROP TABLE  IF EXISTS order_reviews; 
CREATE TABLE order_reviews (
    review_id TEXT,
    order_id TEXT,
    review_score INT,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date TEXT,
    review_answer_timestamp TEXT
    );

-- change directory to where the data is stored
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/olist_order_reviews_dataset.csv' 
INTO TABLE order_reviews 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.order_reviews;


-- (9) read product translations
DROP TABLE  IF EXISTS product_translation; 
CREATE TABLE product_translation (
    product_category_name TEXT,
    product_category_name_english  TEXT   
    );

-- change directory to where you store the data
LOAD DATA LOCAL INFILE 'C:/Users/Desktop/Bus Int/Business Intelligence/Lab8 assignment/product_category_name_translation.csv' 
INTO TABLE product_translation 
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES;

SELECT COUNT(*) 
FROM sql_olist.product_translation;


Exploring the data, creating new tables.

In [ ]:
USE sql_olist;

-- (1) Explore the data, e.g.:

-- what deparments exists?
SELECT DISTINCT product_category_name
FROM products;

-- it is not in English, we can connect it to the translation
SELECT DISTINCT p.product_category_name, t.product_category_name_english
FROM products p


LEFT JOIN product_translation t
ON p.product_category_name = t.product_category_name;

-- how many products by deparment?
SELECT t.product_category_name_english, COUNT(*) AS number
FROM products p
LEFT JOIN product_translation t
ON p.product_category_name = t.product_category_name
GROUP BY t.product_category_name_english
ORDER by number DESC;

-- what if want to know the orders per deparment
SELECT t.product_category_name_english,
COUNT(o.order_id) AS number
FROM orders o
LEFT JOIN order_items i
USING (order_id)
LEFT JOIN products p
USING (product_id) 
LEFT JOIN product_translation t
ON p.product_category_name = t.product_category_name
GROUP BY t.product_category_name_english
ORDER by number DESC;

-- what if we want to select data from a particular deparment (e.g., perfumery)?
SELECT t.product_category_name_english,
o.order_id,
o.customer_id,
o.order_purchase_timestamp,
i.order_item_id,
i.seller_id,
i.price,
i.freight_value,
p.product_category_name,
p.product_weight_g
FROM orders o
LEFT JOIN order_items i
USING (order_id)
LEFT JOIN products p
USING (product_id) 
LEFT JOIN product_translation t
ON p.product_category_name = t.product_category_name
WHERE t.product_category_name_english = 'fashion_shoes';


-- save it as a view or table
DROP table data_to_work;
CREATE TABLE data_to_work
SELECT t.product_category_name_english,
o.order_id,
o.customer_id,
o.order_purchase_timestamp,
i.seller_id,
i.price,
i.freight_value,
p.product_id,
p.product_weight_g
FROM orders o
LEFT JOIN order_items i
USING (order_id)
LEFT JOIN products p
USING (product_id) 
LEFT JOIN product_translation t
ON p.product_category_name = t.product_category_name
WHERE t.product_category_name_english = 'fashion_shoes';


-- you can join this new table with other data now

SELECT  d.*,
    MONTHNAME(order_purchase_timestamp) AS month_name,
    ROUND(price + freight_value, 2) AS price_total,
c.customer_zip_code_prefix,
c.customer_city,
c.customer_state,
r.review_score,
p.payment_type,
o.order_status,
orders.order_purchase_timestamp,
o.order_approved_at,
o.order_delivered_carrier_date,
o.order_delivered_customer_date,
o.order_estimated_delivery_date
FROM data_to_work d
LEFT JOIN order_payments p
USING (order_id)
LEFT JOIN customers c
USING (customer_id)
LEFT JOIN order_reviews r
USING (order_id)
LEFT JOIN orders o
USING (order_id);

-- joining tables and creating new variables
CREATE TABLE BI_shoes_new
SELECT  d.*,
    MONTHNAME(o.order_purchase_timestamp) AS month_name,
    ROUND(price + freight_value, 2) AS price_total,
    c.customer_zip_code_prefix,
    c.customer_city,
    c.customer_state,
    r.review_score,
    r.review_comment_message,
    p.payment_type,
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date
FROM data_to_work d
LEFT JOIN order_payments p
    USING (order_id)
LEFT JOIN customers c
    USING (customer_id)
LEFT JOIN order_reviews r
    USING (order_id)
LEFT JOIN orders o
    USING (order_id);


Error: Expected 0 bindings, saw 3

In [2]:
SELECT * FROM 'BI_shoes_new.csv'

,product_category_name_english,order_id,customer_id,order_purchase_timestamp,seller_id,price,freight_value,product_id,product_weight_g,month_name,price_total,customer_zip_code_prefix,customer_city,customer_state,review_score,review_comment_message,payment_type,order_status,order_purchase_timestamp_1,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,fashion_shoes,9f882c1f904b7dd156572ada58d7a7ae,843ff05b30ce4f75b6170b39c78a8ee5,2017-11-13 10:07:36+00:00,7142540dd4c91e2237acb7e911c4eba2,84.9,17.84,ca92f5c2276bfd491db575cd9de5a70d,800,November,102.74,95595,cidreira,RS,5,Gostei doproduto comprado obrigado.,credit_card,delivered,2017-11-13 10:07:36+00:00,2017-11-13 10:16:29+00:00,2017-12-01 23:19:42+00:00,2017-12-11 17:38:06+00:00,2017-12-22 00:00:00+00:00
1,fashion_shoes,1cb6cfd9e6306cfe607017391834999a,05aa1cdcd7c5f9e297b3b5d89ae5e1df,2017-04-29 20:09:24+00:00,7142540dd4c91e2237acb7e911c4eba2,74.9,28.43,6f2c108731ef77738da45855e737c1bd,1400,April,103.33,60713,fortaleza,CE,5,Obrigado vocês estão de parabéns produtos exce...,credit_card,delivered,2017-04-29 20:09:24+00:00,2017-04-29 20:22:41+00:00,2017-05-11 14:57:53+00:00,2017-05-24 12:59:33+00:00,2017-06-27 00:00:00+00:00
2,fashion_shoes,1cb6cfd9e6306cfe607017391834999a,05aa1cdcd7c5f9e297b3b5d89ae5e1df,2017-04-29 20:09:24+00:00,7142540dd4c91e2237acb7e911c4eba2,74.9,30.47,1c99a6128a79a5fb5a34c1c6b3be9067,900,April,105.37,60713,fortaleza,CE,5,Obrigado vocês estão de parabéns produtos exce...,credit_card,delivered,2017-04-29 20:09:24+00:00,2017-04-29 20:22:41+00:00,2017-05-11 14:57:53+00:00,2017-05-24 12:59:33+00:00,2017-06-27 00:00:00+00:00
3,fashion_shoes,305b0101ce8f8fd2d7b32f7480340018,06872c61a13a4d027504dd0c15d43b72,2017-04-30 18:34:22+00:00,827f8f69dfa529c561901c4f2e0f332f,29.9,14.52,de607575b49bac3e75ff4e11bd3d3c11,200,April,44.42,12530,cunha,SP,5,None,credit_card,delivered,2017-04-30 18:34:22+00:00,2017-04-30 18:45:15+00:00,2017-05-05 12:07:58+00:00,2017-05-16 08:57:50+00:00,2017-05-25 00:00:00+00:00
4,fashion_shoes,af42887f85d980291a066176c4d13709,8f0b436a1be5cd29776f9ddb3bef133a,2017-06-18 19:51:54+00:00,827f8f69dfa529c561901c4f2e0f332f,35.9,14.10,f39288f58793ce5e6985b3f7832af88c,300,June,50.00,07786,cajamar,SP,2,Produto foi entregue com muita demora\r\n,boleto,delivered,2017-06-18 19:51:54+00:00,2017-06-20 10:35:28+00:00,2017-06-27 16:37:54+00:00,2017-07-04 14:41:48+00:00,2017-07-11 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,fashion_shoes,cc62c4c4dc7f0bef11fd8848aebb5da7,dea0282651f3ab3a8621120d97f9bab7,2017-09-16 18:33:08+00:00,7142540dd4c91e2237acb7e911c4eba2,69.9,17.74,990776b3e410ccb3cc725fddb0d20db9,550,September,87.64,39915,mata verde,MG,4,"vendedor confiavel, o pedido chegou no prazo c...",credit_card,delivered,2017-09-16 18:33:08+00:00,2017-09-16 18:50:21+00:00,2017-09-22 20:13:49+00:00,2017-10-03 16:29:08+00:00,2017-10-17 00:00:00+00:00
274,fashion_shoes,cc62c4c4dc7f0bef11fd8848aebb5da7,dea0282651f3ab3a8621120d97f9bab7,2017-09-16 18:33:08+00:00,7142540dd4c91e2237acb7e911c4eba2,69.9,17.74,2880c8710857e36b294c7b9380b2e6cd,1500,September,87.64,39915,mata verde,MG,4,"vendedor confiavel, o pedido chegou no prazo c...",credit_card,delivered,2017-09-16 18:33:08+00:00,2017-09-16 18:50:21+00:00,2017-09-22 20:13:49+00:00,2017-10-03 16:29:08+00:00,2017-10-17 00:00:00+00:00
275,fashion_shoes,f5b3a06b2ac88029188565f5e8bf4347,66aa9db44cb942fec96bd377024eca05,2017-11-14 07:50:39+00:00,5dceca129747e92ff8ef7a997dc4f8ca,69.9,9.34,63a033b030ff3da0b4b4eb3043c503d1,600,November,79.24,07096,guarulhos,SP,1,None,voucher,canceled,2017-11-14 07:50:39+00:00,2017-11-14 08:06:24+00:00,NaT,NaT,2017-11-28 00:00:00+00:00
276,fashion_shoes,f695e0534456bafac8cda166dc5fd0e4,04fa959a05c178cef48c006e12af8024,2017-08-29 22:25:09+00:00,7142540dd4c91e2237acb7e911c4eba2,84.9,13.61,f45d3b91af1103db9a21dc9d11cb9223,750,August,98.51,07131,guarulhos,SP,3,None,credit_card,delive